##  Talent Scheduling 

1. Welche Entscheidungen / Handlungen / Aktionen sind möglich?
- eine Beschreibung des aktuellen **Zustands**:
  - TSP: die bisherige Tour
  - Rucksackproblem: das bisher eingepackte Gewicht
  - *Talent Scheduling*: die Reihenfolge der zu drehenden Szenen 
- die "Regeln" / Restriktionen, die beschreiben, **welche Handlungen in einem Zustand zulässig sind**
  - TSP: kein erneuter Besuch einer bereits besuchten Stadt
  - Rucksackproblem: Gegenstand einpacken geht nur, wenn das Gesamtgewicht nicht überschritten wird
  - *Talent Scheduling*: eine Szene wird nur einmal gedreht
 
2. Welche der möglichen Entscheidungen sollte gewählt werden?
- z.B. der Wert der neuen Teillösung:
  - TSP: Distanz der (Teil-)Tour nach Hinzunahme des Nacharn
  - Rucksackproblem: Gesamtwert der Lösung nach Hinzunahme eines Elements
  - *Talent Scheduling*: Die Szenen, die am meisten Kosten, werden zuerst gedreht(= die Szenen absteigend nach Kosten sortieren)

In [3]:
#IMPORTs
import numpy as np
import matplotlib.pyplot as plt

#numba
from numba import njit, jit
from numba.typed import List

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

In [90]:
class ts_udp:
    """
    UDP (User-Defined Problem) class for the Talent Scheduling problem of the 
    from the course AuD(Team02[LG,DSc,RK,DSw])

    Summary
    ========
    The talent scheduling problem can be described as follows. A film producer needs 
    to schedule the scenes of his/her movie on a given location. Each scene has a duration (the days it takes to shoot it) 
    and requires some subset of the cast to be on location. The cast are paid for each day they are required to be on 
    location from the day the first scene they are in is shot, to the day the last scene they are in is shot, even though 
    some of those days they might not be required by the scene currently being shot (i.e., they will be on location waiting
    for the next scene they are in to be shot). Each cast member has a different daily salary. The aim of the film producer
    is to order the scenes in such a way as to minimize the salary cost of the shooting. 

    """

    def __init__(self, file_path):
        self.file_path = file_path
        self.name = ""
        self.num_scenes = 0
        self.num_actors = 0
        self.scenes = []
        self.actor_apprs = []
        self.actor_cost = []
        self.scene_duration = []
        self._load_instance()

    def _load_instance(self):
        """
        Load the instance from the dt-file.
        """
        with open(self.file_path, 'r') as file:
            lines = file.readlines()
            self.name = lines[0].strip()  # First line is the name of the data file
            self.num_scenes = int(lines[1].strip())  # Second line is the number of scenes
            self.num_actors = int(lines[2].strip())  # Third line is the number of actors
            
            # Reading actor data
            actor_data = lines[4:4 + self.num_actors]
            for actor_line in actor_data:
                parts = actor_line.strip().split()
                appearance = list(map(int, parts[:-1]))  # convert each part except last to int
                cost = int(parts[-1])  # Last part is the cost
                self.actor_apprs.append(appearance)
                self.actor_cost.append(cost)

            # Reading scene duration
            self.scene_duration = list(map(int, lines[5 + self.num_actors].strip().split()))



    ################
    # Public Methods
    ################

    def info(self):
        print(f"File name: {self.name}")
        print(f"Number of scenes: {self.num_scenes}, Number of actors: {self.num_actors}")
        print(f"Actor appearances: {self.actor_apprs}")
        print(f"Actor costs: {self.actor_cost}")
        print(f"Scene durations: {self.scene_duration}")

    #@njit
    def scene_cost(self):
        # Konvertieren der Liste in ein numpy 2D-Array, für Recheneffizienz
        array_actor_apprs = np.array(self.actor_apprs)
        
        # Array, mit dem multipliziert werden soll
        array_actor_cost = np.array(self.actor_cost)
        
        # Durchführen der Multiplikation mittels Broadcasting für effizienz
        zwischen_ergebnis = array_actor_apprs * array_actor_cost[:, np.newaxis]
        
        array_scene_duration = np.array(self.scene_duration)
        
        # Durchführen der zweiten Multiplikation mittels Broadcasting
        ergebnis = zwischen_ergebnis * array_scene_duration
        
        
        sk=0
        for e in ergebnis:
            sk += e
        #print(f'die Szenen-Kosten {sk}')
        return list(sk)
    
    #@njit
    def ts_greedy(self):
        sk = self.scene_cost()
        return [i for i in sorted(range(len(sk)), key=lambda x: sk[x], reverse=True)]

    def zero_degree(self, scene_order):
        zero_counts = [0] * len(self.scene_duration)
        for actor_presence in self.actor_apprs:
            for i in range(len(scene_order) - 1):
                if actor_presence[scene_order[i]] == 1:
                    zero_count = 0
                    for j in range(i + 1, len(scene_order)):
                        if actor_presence[scene_order[j]] == 0:
                            zero_count += 1
                    zero_counts[scene_order[i]] += zero_count
        return zero_counts

    def ts_greedy_plus(self, start_scene):
        scene_order = [start_scene]
        available_scenes = list(range(len(self.scene_duration)))
        available_scenes.remove(start_scene)

        while available_scenes:
            zero_counts = self.zero_degree(scene_order)
            next_scene = min(available_scenes, key=lambda x: zero_counts[x])
            scene_order.append(next_scene)
            available_scenes.remove(next_scene)
        
        return scene_order
                


    def reorder_lists(self,order):
        actor_apprs = self.actor_apprs
        scene_durations = self.scene_duration
        
        # Neu ordnen der Szenendauern und Auftritte
        new_scene_durations = [scene_durations[i] for i in order]

        new_actor_apprs = [[actor_apprs[j][i] for i in order] for j in range(len(actor_apprs))]
        
        return new_actor_apprs, new_scene_durations


    def extra(self, order):
        actor_apprs, scene_durations = self.reorder_lists(order)
        extra_cost = 0
        for list_index, act_list in enumerate(actor_apprs):
            mod_len = len(act_list)
            while mod_len > 0 and act_list[mod_len - 1] == 0:
                mod_len -= 1

            i = 0
            while i < mod_len - 1:
                if act_list[i] == 1 and act_list[i + 1] == 0:
                    while i + 1 < mod_len and act_list[i + 1] != 1:
                        extra_cost += self.actor_cost[list_index] * scene_durations[i + 1]
                        i += 1
                i += 1
        
        return extra_cost
    


def score(extra):
    sl = film.scene_cost()
    rawcost = sum(sl)
    mincost = rawcost + extra 
    return mincost #+ unnötige Kosten(extra)
    

In [91]:
film = ts_udp(file_path = 'data_prj/film116.dat')
film.info()

print(100 * "~")

print(f'die Szenen-Kosten;    {film.scene_cost()}')
print(f'die Greedy-Lösung;    {film.ts_greedy()}')

print("die 'optimale' Lösung;[18, 17, 14, 3, 0, 16, 1, 12, 13, 6, 4, 10, 2, 9, 15, 5, 7, 11, 8]")

File name: film116.dat
Number of scenes: 19, Number of actors: 8
Actor appearances: [[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]]
Actor costs: [10, 4, 5, 5, 5, 40, 4, 20]
Scene durations: [1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
die Szenen-Kosten;    [40, 19, 49, 30, 9, 45, 5, 10, 10, 53, 15, 9, 18, 9, 20, 40, 20, 25, 5]
die Greedy-Lösung;    [9, 2, 5, 0, 15, 3, 17, 14, 16, 1, 12, 10, 7, 8, 4, 11, 13, 6, 18]
die 'optimale' Lösung;[18, 17, 14, 3, 0, 16

In [92]:
print(f'rowcost; {sum(film.scene_cost())}')
print(f'extra; {film.extra(order = film.ts_greedy_plus(0))}')
print(f'mincost; {score(film.extra(film.ts_greedy_plus(0)))}')

rowcost; 431
extra; 1021
mincost; 1452


In [45]:
%%time
film.reorder_lists()

CPU times: user 72 µs, sys: 13 µs, total: 85 µs
Wall time: 88.2 µs


([[0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
  [0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0],
  [0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
  [1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0],
  [0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1])

In [61]:
%%time
film.ts_greedy_plus(0)
print(len(film.scene_duration))

19
CPU times: user 265 µs, sys: 47 µs, total: 312 µs
Wall time: 306 µs


### Instanzen 'small & small02'

```
small
6
4

1 0 0 0 1 1   1
1 1 0 1 0 0   1
0 1 1 0 1 0   1
0 0 1 1 0 1   1

1 2 3 4 5 6

============================================

small2
6
4

1 1 0 0 1 1   1
1 1 0 1 0 0   1
0 1 1 1 1 0   1
0 0 1 1 0 1   1

1 2 3 4 5 6
```

```
========================================== small
Solution:    3    1    0    2    4    5
Solution:    3    1    0    4    2    5
Solution:    3    1    2    0    4    5
Solution:    3    2    1    0    4    5
Solution:    4    2    1    0    3    5
Solution:    4    2    1    3    0    5
#Solutions=6
mincost = 54, rawcost = 42, extra = 12, bound = 134546788
ccc = 63,loop = 192
reuse = 124, reenter = 0, definite = 0, lookahead = 0, diffuse = 0, clump = 0
FINAL
  5   4   2   0   1   3 
------------------------
  X   X   -   X   .   . |  1
  .   .   .   X   X   X |  1
  .   X   X   -   X   . |  1
  X   -   X   -   -   X |  1
------------------------
  6   5   3   1   2   4 
------------------------
 12  15   9   4   6   8  = 54
------------------------
 12  10   6   2   4   8 
TIME = 0 ms
FINALDATA 6 4 6 54 63 124 0 0 0 tv small


========================================== small2
Solution:    4    0    1    3    2    5
Solution:    4    1    0    3    2    5
#Solutions=2
mincost = 56, rawcost = 48, extra = 8, bound = 134546788
ccc = 63,loop = 192
reuse = 124, reenter = 0, definite = 0, lookahead = 0, diffuse = 0, clump = 0
FINAL
  5   2   3   1   0   4 
------------------------
  X   -   -   X   X   X |  1
  .   .   X   X   X   . |  1
  .   X   X   X   -   X |  1
  X   X   X   .   .   . |  1
------------------------
  6   3   4   2   1   5 
------------------------
 12   9  16   6   3  10  = 56
------------------------
 12   6  12   6   2  10 
TIME = 0 ms
FINALDATA 6 4 6 56 63 124 0 0 0 tv small2

========================================== film116
.
.
.

mincost = 541, rawcost = 431, extra = 110, bound = 134546788
ccc = 524287,loop = 4980736
reuse = 4456431, reenter = 0, definite = 0, lookahead = 0, diffuse = 0, clump = 0
FINAL
 18  17  14   3   0  16   1  12  13   6   4  10   2   9  15   5   7  11   8 
----------------------------------------------------------------------------
  .   .   .   X   X   X   X   X   .   .   .   .   .   .   .   .   .   .   . |  10
  .   .   .   .   .   .   .   X   X   -   -   -   -   X   .   .   .   .   . |  4
  .   .   .   .   X   X   X   -   X   X   X   X   X   .   .   .   .   .   . |  5
  .   .   .   .   .   .   .   .   .   .   .   X   -   X   -   -   X   X   X |  5
  X   X   -   -   X   X   -   -   -   -   -   X   -   -   -   X   X   .   . |  5
  .   .   .   .   .   .   .   .   .   .   .   .   X   X   X   X   .   .   . |  40
  .   .   .   .   .   .   X   X   -   -   X   -   X   X   -   -   -   X   . |  4
  .   X   X   X   X   .   .   .   .   .   .   .   .   .   .   .   .   .   . |  20
----------------------------------------------------------------------------
  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   2 
----------------------------------------------------------------------------
  5  25  25  35  40  20  24  28  18  18  18  23  63  58  54  54  14   9  10  = 541
----------------------------------------------------------------------------
  5  25  20  30  40  20  19  18   9   5   9  15  49  53  40  45  10   9  10 
TIME = 776 ms
FINALDATA 19 8 19 541 524287 4456431 0 776 0 tv film116.dat